In [1]:
# Import required libraries
import pickle
import copy
import pathlib
import dash
import math
import datetime as dt
import pandas as pd
from dash.dependencies import Input, Output, State, ClientsideFunction
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

from jupyter_dash import JupyterDash

In [2]:
sdmx_url = 'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/.{}....?format=csv'

In [9]:
codes = [
    "EDU_SDG_STU_L2_MATH",
    "EDU_SDG_STU_L2_READING",
    "EDU_SDG_STU_L1_GLAST_MATH",
    "EDU_SDG_STU_L1_G2OR3_MATH",
    "EDU_SDG_STU_L1_GLAST_READING",
    "EDU_SDG_STU_L1_G2OR3_READING",
    "EDUNF_NERA_L2",
    "EDUNF_NERA_L1_PT",
    "EDU_SDG_GER_L01",
    "EDUNF_LR_L02",
    "EDUNF_ROFST_L2",
    "EDU_SDG_QUTP_L02",
    "EDU_SDG_QUTP_L1",
    "EDU_SDG_QUTP_L2",
    "EDU_SDG_QUTP_L3",
    "EDU_SDG_TRTP_L02",
    "EDU_SDG_TRTP_L1",
    "EDU_SDG_TRTP_L2",
    "EDU_SDG_TRTP_L3",
]

data = pd.DataFrame()
inds = set()
for ind in codes:
    sdmx = pd.read_csv(sdmx_url.format(ind))
    sdmx["CODE"] = ind
    data = data.append(sdmx)

In [10]:
# Create controls
county_options = [
    {"label": str(country), "value": str(country)} for country in data['Geographic area'].unique()
]

In [11]:
county_options

[{'label': 'Albania', 'value': 'Albania'},
 {'label': 'Armenia', 'value': 'Armenia'},
 {'label': 'Azerbaijan', 'value': 'Azerbaijan'},
 {'label': 'Bosnia and Herzegovina', 'value': 'Bosnia and Herzegovina'},
 {'label': 'Bulgaria', 'value': 'Bulgaria'},
 {'label': 'Belarus', 'value': 'Belarus'},
 {'label': 'Czechia', 'value': 'Czechia'},
 {'label': 'Estonia', 'value': 'Estonia'},
 {'label': 'Georgia', 'value': 'Georgia'},
 {'label': 'Croatia', 'value': 'Croatia'},
 {'label': 'Hungary', 'value': 'Hungary'},
 {'label': 'Kyrgyzstan', 'value': 'Kyrgyzstan'},
 {'label': 'Kazakhstan', 'value': 'Kazakhstan'},
 {'label': 'Lithuania', 'value': 'Lithuania'},
 {'label': 'Latvia', 'value': 'Latvia'},
 {'label': 'Republic of Moldova', 'value': 'Republic of Moldova'},
 {'label': 'Montenegro', 'value': 'Montenegro'},
 {'label': 'North Macedonia', 'value': 'North Macedonia'},
 {'label': 'Poland', 'value': 'Poland'},
 {'label': 'Romania', 'value': 'Romania'},
 {'label': 'Serbia', 'value': 'Serbia'},
 {'

In [12]:
years =[i for i in range(2010, 2020)]

indicators = data['Indicator'].unique()

{index: str(year) for index,year in enumerate(years)}

{0: '2010',
 1: '2011',
 2: '2012',
 3: '2013',
 4: '2014',
 5: '2015',
 6: '2016',
 7: '2017',
 8: '2018',
 9: '2019'}

In [37]:
qutp = data[
    data["CODE"].isin([
            "EDU_SDG_QUTP_L02",
            "EDU_SDG_QUTP_L1",
            "EDU_SDG_QUTP_L2",
            "EDU_SDG_QUTP_L3",
        ])
].groupby(["Geographic area", "TIME_PERIOD"], sorted=True
).agg({"OBS_VALUE": "mean"}
).reset_index()
qutp['Indicator'] = 'Proportion of teachers qualified according to national standards'

trtp = data[
    data["CODE"].isin([
            "EDU_SDG_TRTP_L02",
            "EDU_SDG_TRTP_L1",
            "EDU_SDG_TRTP_L2",
            "EDU_SDG_TRTP_L3",
        ])
].groupby(["Geographic area", "TIME_PERIOD"], sorted=True
).agg({"OBS_VALUE": "mean"}
).reset_index()
trtp['Indicator'] = 'Proportion of teachers who have received at least the minimum organized teacher training'

df = qutp.append(trtp)
df
# from mapbox import Geocoder

# mapbox_access_token = "pk.eyJ1IjoiamNyYW53ZWxsd2FyZCIsImEiOiJja2NkMW02aXcwYTl5MnFwbjdtdDB0M3oyIn0.zkIzPc4NSjLZvrY-DWrlZg"

# geocoder = Geocoder(access_token=mapbox_access_token)
# def geocode_address(address):
#     """Geocode street address into lat/long."""
#     response = geocoder.forward(address)
#     coords = response.json()['features'][0]['center']
#     # coords = coords.replace(']', '')
#     # coords = coords.replace('[', '')
#     coords

# data.apply(lambda row: pd.Series(geocode_address(row['Geographic area'])), axis=1)


,Geographic area,TIME_PERIOD,OBS_VALUE,Indicator
0,Albania,2013,80.115055,Proportion of teachers qualified according to ...
1,Albania,2014,83.318565,Proportion of teachers qualified according to ...
2,Albania,2015,85.794593,Proportion of teachers qualified according to ...
3,Albania,2016,88.401573,Proportion of teachers qualified according to ...
4,Albania,2017,90.533308,Proportion of teachers qualified according to ...
...,...,...,...,...
145,Uzbekistan,2009,100.000000,Proportion of teachers who have received at le...
146,Uzbekistan,2010,100.000000,Proportion of teachers who have received at le...
147,Uzbekistan,2011,100.000000,Proportion of teachers who have received at le...
148,Uzbekistan,2017,96.103695,Proportion of teachers who have received at le...
